### Use Word2Vec to train your own model on a dataset.

1) **Optional** - Find your own dataset of documents to train you model on. You are going to need a lot of data, so it's probably not realistic to scrape data for this assignment given the time constraints that we're working under. Try to find a dataset that has > 5000 documents.

- If you can't find a dataset to use try this one: <https://www.kaggle.com/c/quora-question-pairs>

2) Clean/Tokenize the documents.

3) Vectorize the model using Word2Vec and explore the results using each of the following at least one time:

- your_model.wv.most_similar()
- your_model.wv.similarity()
- your_model.wv.doesn't_match()

In [13]:
!pip install -U gensim

    100% |████████████████████████████████| 24.2MB 1.1MB/s ta 0:00:011    88% |████████████████████████████▏   | 21.3MB 3.1MB/s eta 0:00:01
    100% |████████████████████████████████| 40kB 4.0MB/s ta 0:00:011
    100% |████████████████████████████████| 133kB 1.7MB/s ta 0:00:01
    100% |████████████████████████████████| 5.4MB 1.9MB/s ta 0:00:011
  Stored in directory: /home/mark/.cache/pip/wheels/f7/a6/ff/9ab5842c14e50e95a06a4675b0b4a689c9cab6064dac2b01d0
  Stored in directory: /home/mark/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file
awscli 1.16.125 has requirement botocore==1.12.115, but you'll have botocore 1.12.123 which is incompatible.
  Found existing installation: botocore 1.12.115
    Uninstalling botocore-1.12.115:
      Successfully uninstalled botocore-1.12.115


In [ ]:
!wget "http://archive.ics.uci.edu/ml/machine-learning-databases/20newsgroups-mld/20_newsgroups.tar.gz"


In [ ]:
!tar -xzf 20_newsgroups.tar.gz

In [39]:
import re
import io
import os
from os import listdir
from os.path import isfile, join
ndir = '/home/mark/lambda/DS-Unit-4-Sprint-2-NLP/module4-Word-Embeddings/20_newsgroups/'
# files = [f for f in listdir(ndir) if isfile(join(ndir, f))]
from os import walk
import pandas as pd
import gensim
from gensim.models.word2vec import Word2Vec
from nltk.tokenize import word_tokenize
import string

In [40]:
files = [val for sublist in [[os.path.join(i[0], j) for j in i[2]] for i in os.walk(ndir)] for val in sublist]

In [41]:
len(files)

19997

In [42]:
da = []
for f in files:
    m = re.search('((?<=/)([^/])+(?=/\d+$))',f)
#     print(m.groups()[0],f)
    with open(f,'r',encoding="ISO-8859-1") as fh:
        lines = fh.readlines()
        L = 0
        for i in range(10,len(lines)):
            if (len(lines[i]) < 6) or ("From article" in lines[i]) or ("Lines:" in lines[i]):
#                 print(i,lines[i])
                continue
            else:
                L = i
                break
#         print('L',L)        
        da.append({'text': lines[L:], 'ngName': m.groups()[0]})
print(len(da))

19997


In [43]:
df = pd.DataFrame(da)

In [44]:
df.head()

,ngName,text
0,sci.electronics,[> erickson@azure.nmt.edu (Alan Erickson) writ...
1,sci.electronics,"[HI, I was wondering if anyone would be able t..."
2,sci.electronics,[In article <1993Apr23.105152.20155@news.cs.tu...
3,sci.electronics,"[>Dean Anneser (anneser@pwa-b.uucp) wrote:\n, ..."
4,sci.electronics,"[oops, that's KNX 1070. KNBR is up in 'Frisco,..."


In [45]:
df.ngName.unique()

array(['sci.electronics', 'comp.graphics', 'soc.religion.christian',
       'misc.forsale', 'rec.sport.baseball', 'alt.atheism', 'rec.autos',
       'talk.politics.mideast', 'sci.crypt', 'sci.med',
       'comp.os.ms-windows.misc', 'comp.sys.mac.hardware',
       'rec.motorcycles', 'talk.religion.misc', 'talk.politics.guns',
       'comp.sys.ibm.pc.hardware', 'comp.windows.x', 'rec.sport.hockey',
       'talk.politics.misc', 'sci.space'], dtype=object)

In [57]:
import nltk
# nltk.download('all')

In [51]:

# print(string.punctuation)

def process_text(text):
  """Remove punctuation, lowercase, and tokenize text."""
  # TODO: check for special cases like "I'll"
  text = "".join([char.lower() for char in text
                  if char not in string.punctuation])
  return word_tokenize(text)
sentences = []
for texta in df.text:
    for text in texta:
        t = process_text(text)
        if len(t) == 0:  #len(t) <= 1 and t != 'i':
            continue
        if len(sentences) < 10:
            print('t |',t,'|',len(t))
        sentences.append(t)
print(sentences[:10])

t | ['ericksonazurenmtedu', 'alan', 'erickson', 'writes'] | 4
t | ['im', 'trying', 'to', 'bring', 'in', '8', 'bits', 'to', 'a', 'pc', 'and', 'would', 'like'] | 13
t | ['to', 'use', 'interruptdriven', 'routines', 'without', 'buying', 'an', 'io'] | 8
t | ['board', 'or', 'making', 'a', 'new', 'port', 'where', 'can', 'i', 'bring', 'in', 'these'] | 12
t | ['bits', 'lpt', 'seems', 'to', 'have', 'only', 'a', 'few', 'inputs', 'but', 'ive', 'heard'] | 12
t | ['rumours', 'that', 'some', 'lpts', 'have', 'bidirectional', 'lines', 'anybody'] | 8
t | ['know', 'fer', 'sure', 'if', 'any', 'bid', 'lpts', 'which', 'boards', 'have', 'them'] | 11
t | ['ill', 'be', 'running', 'a', 'new', '386dx33'] | 6
t | ['yes', 'it', 'is', 'possible', 'im', 'making', 'a', '7', 'stepper', 'controller', 'board'] | 11
t | ['with', '7', 'digital', 'inputs', 'and', 'up', 'to', '18', 'digital', 'outputs', 'from', 'the', 'port'] | 13
[['ericksonazurenmtedu', 'alan', 'erickson', 'writes'], ['im', 'trying', 'to', 'bring', 'in', 

In [52]:
news_model = Word2Vec(sentences, min_count=3, size=200, window=2)

In [54]:
news_model.wv.most_similar('windows')

[('dos', 0.7775426506996155),
 ('os2', 0.7130886316299438),
 ('xv', 0.7054568529129028),
 ('apps', 0.7029247879981995),
 ('mac', 0.6843228340148926),
 ('driver', 0.6786565780639648),
 ('motif', 0.6751431226730347),
 ('microsoft', 0.6671056151390076),
 ('desktop', 0.6580813527107239),
 ('drivers', 0.6558448076248169)]

In [56]:
news_model.wv.most_similar('linux')

[('apps', 0.7872177362442017),
 ('pcs', 0.7800400257110596),
 ('msdos', 0.7620121240615845),
 ('mswindows', 0.7596473693847656),
 ('xwindows', 0.7527232766151428),
 ('x11r4', 0.7419207096099854),
 ('unix', 0.7404413223266602),
 ('tcpip', 0.7356176376342773),
 ('sgi', 0.7253162860870361),
 ('x11r5', 0.7238607406616211)]

In [58]:
news_model.wv.similarity('controller', 'board')

0.7060164

In [59]:
news_model.wv.similarity('input', 'port')

0.7121664

In [60]:
news_model.wv.similarity('output', 'port')

0.75221884

In [61]:
news_model.wv.similarity('output', 'input')

0.85966665

In [65]:
news_model.wv.doesnt_match(['windows', 'linux', 'like'])

'like'

In [66]:
news_model.wv.doesnt_match(['bidirectional', 'port', 'input', 'output'])

'bidirectional'

### Stretch Goals:

1) Use Doc2Vec to train a model on your dataset, and then provide model with a new document and let it find similar documents.

2) Download the pre-trained word vectors from Google. Access the pre-trained vectors via the following link: https://code.google.com/archive/p/word2vec

Load the pre-trained word vectors and train the Word2vec model

Examine the first 100 keys or words of the vocabulary

Outputs the vector representation for a select set of words - the words can be of your choice

Examine the similarity between words - the words can be of your choice

For example:

model.similarity('house', 'bungalow')

model.similarity('house', 'umbrella')

In [ ]:
df.head()